In [1]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras import layers
import time

from IPython import display

## Mnsit Data Loading

In [3]:
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data() # Load the dataset
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32') # Reshape the images to (28, 28, 1)
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 11s 1us/step
